In [ ]:
import pandas as pd
import numpy as np
import re

# Load the datasets
VGSales_union = pd.read_csv("VGSales_Union.csv", na_values=["N/A", "NA"])
VGSales_intersect = pd.read_csv("VGSales_Intersect.csv", na_values=["N/A", "NA"])

def cleaning_data(df):
    # Trim white space for 'summary' and 'publisher'
    if "summary" in df.columns:
        df["summary"] = df["summary"].str.strip()
    if "publisher" in df.columns:
        df["publisher"] = df["publisher"].str.strip()

    # Format the Metacritic release date to only have month and year
    if "release_date" in df.columns:
        df["release_date"] = pd.to_datetime(df["release_date"], format="%B %d, %Y", errors='coerce')
        df["release_date"] = df["release_date"].dt.strftime("%B %Y")

    # Clean VGChartz Release_Date to month-year format
    if "Release_Date" in df.columns:
        df["Release_Date"] = df["Release_Date"].str.replace(r"(st|nd|rd|th)", "", regex=True)
        df["Release_Date"] = pd.to_datetime(df["Release_Date"], format="%d %b %y", errors='coerce')
        df["Release_Date"] = df["Release_Date"].dt.strftime("%B %Y")

    # Convert sales columns to numeric (millions to actual values)
    if "NA_Sales" in df.columns:
        df["NA_Sales"] = pd.to_numeric(df["NA_Sales"].str.replace("m", ""), errors='coerce') * 1_000_000
    if "Total_Sales" in df.columns:
        df["Total_Sales"] = pd.to_numeric(df["Total_Sales"].str.replace("m", ""), errors='coerce') * 1_000_000

    # Replace missing Metacritic release dates with VGChartz release dates
    if "release_date" in df.columns and "Release_Date" in df.columns:
        df["release_date"] = np.where(df["release_date"].isna(), df["Release_Date"], df["release_date"])

    # Set 'games', 'platform', and 'publisher' as categorical variables
    if "games" in df.columns:
        df["games"] = df["games"].astype("category")
    if "platform" in df.columns:
        df["platform"] = df["platform"].astype("category")
    if "publisher" in df.columns:
        df["publisher"] = df["publisher"].astype("category")

    # Rename columns
    df = df.rename(columns={
        "games": "game_title",
        "NA_Sales": "na_sales",
        "Total_Sales": "total_sales"
    })

    # Drop unnecessary columns
    drop_columns = ["X", "X.1", "PAL_Sales", "JP_Sales", "Other_Sales", "Release_Date"]
    df = df.drop(columns=[col for col in drop_columns if col in df.columns], errors='ignore')

    return df

# Check for duplicated data in both datasets
print("Duplicated rows in VGSales_intersect:", VGSales_intersect.duplicated().sum())
print("Duplicated rows in VGSales_union:", VGSales_union.duplicated().sum())

# Clean both datasets
VGSales_intersect = cleaning_data(VGSales_intersect)
VGSales_union = cleaning_data(VGSales_union)

# Save the cleaned datasets
VGSales_intersect.to_csv("VGSales_Intersect_cleaned.csv", index=False)
VGSales_union.to_csv("VGSales_Union_cleaned.csv", index=False)

